In [1]:
import pandas as pd
import numpy as np

In [2]:
HED_df = pd.read_excel('Meter 1 - Head End Read Data.xlsx', sheet_name='Meter Reads HES')
MDMS_df = pd.read_excel('Meter 1 - MDMS Read Data.xlsx', sheet_name='MDMS Meter reads')

In [3]:
HED_df.head()

,Intervals (UTC),Analysis,Interval Status,Consumption (Liters) - Status,Consumption (Liters) - Usage,Consumption (Liters) - Reading
0,2025-11-06 16:00:00,NaN,NaN,--,0,0
1,2025-11-06 17:00:00,Potential Fill,NaN,--,--,--
2,2025-11-06 18:00:00,Potential Fill,NaN,--,--,--
3,2025-11-06 19:00:00,Potential Fill,NaN,--,--,--
4,2025-11-06 20:00:00,Potential Fill,NaN,--,--,--


In [4]:
HED_df.tail()

,Intervals (UTC),Analysis,Interval Status,Consumption (Liters) - Status,Consumption (Liters) - Usage,Consumption (Liters) - Reading
424,2025-11-24 08:00:00,NaN,NaN,--,10,6202
425,2025-11-24 09:00:00,NaN,NaN,--,21,6223
426,2025-11-24 10:00:00,NaN,NaN,--,0,6223
427,2025-11-24 11:00:00,NaN,NaN,--,0,6223
428,2025-11-24 12:00:00,NaN,NaN,--,6,6229


In [5]:
HED_df.drop(columns=['Analysis','Interval Status','Consumption (Liters) - Status'], inplace=True)

In [6]:
HED_df.rename(columns={'Intervals (UTC)':'Interval','Consumption (Liters) - Usage':'Usage','Consumption (Liters) - Reading':'Reading'}, inplace=True)

In [7]:
HED_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Interval  429 non-null    datetime64[ns]
 1   Usage     429 non-null    object        
 2   Reading   429 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 10.2+ KB


In [8]:
HED_df['Usage'] = pd.to_numeric(HED_df['Usage'], errors='coerce')
HED_df['Reading'] = pd.to_numeric(HED_df['Reading'], errors='coerce')

In [9]:
HED_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Interval  429 non-null    datetime64[ns]
 1   Usage     425 non-null    float64       
 2   Reading   425 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 10.2 KB


In [10]:
# Check for duplicate intervals
HED_df['Interval'].duplicated().sum()

0

In [11]:
# Check for missing values in interval column
HED_df['Interval'].isnull().sum()

0

In [12]:
HED_df = HED_df.sort_values('Interval')

In [13]:
# Check for missing intervals
expected_count = pd.date_range(start=HED_df['Interval'].min(),end=HED_df['Interval'].max(),freq='H').size
actual_count = HED_df['Interval'].nunique()
print("Expected:", expected_count)
print("Actual:", actual_count)

Expected: 429
Actual: 429


In [14]:
# check for unusual meter readings
HED_df['Usage'].describe()

count    425.000000
mean      14.487059
std       19.939897
min        0.000000
25%        0.000000
50%        0.000000
75%       29.000000
max       90.000000
Name: Usage, dtype: float64

In [15]:
# look for missing values in Usage column
HED_df['Usage'].isna().sum()

4

In [16]:
# Making sure comulative usage is calculated correctly
HED_df["Cumulative_from_72"] = 72 + HED_df["Usage"].cumsum()
HED_df["Reading_diff"] = HED_df["Reading"] - HED_df["Cumulative_from_72"]
HED_df[HED_df["Reading_diff"] != 0]

,Interval,Usage,Reading,Cumulative_from_72,Reading_diff
0,2025-11-06 16:00:00,0.0,0.0,72.0,-72.0
1,2025-11-06 17:00:00,NaN,NaN,NaN,NaN
2,2025-11-06 18:00:00,NaN,NaN,NaN,NaN
3,2025-11-06 19:00:00,NaN,NaN,NaN,NaN
4,2025-11-06 20:00:00,NaN,NaN,NaN,NaN


In [17]:
HED_df.drop(columns=['Cumulative_from_72','Reading_diff'], inplace=True)

In [18]:
HED_df.to_csv('cleaned_HED_data.csv', index=False)